In [1]:
import os, glob, warnings
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
from shapely.ops import unary_union
from fiona.env import Env
from pykrige.ok import OrdinaryKriging
from numpy.linalg import LinAlgError
from scipy.spatial import cKDTree

# ---- Paths
DATA_DIR  = r"C:\Users\krish\Desktop\SpatialCARE\DailyGPKG"
PASIG_SHP = r"C:\Users\krish\Desktop\PhD Class\Shapefile\MM\Pasig\Pasig.shp"
OUT_DIR   = r"C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\uncertainty"
os.makedirs(OUT_DIR, exist_ok=True)

# ---- CRS, fig, grid, kriging
WGS84, UTM51 = "+proj=longlat +datum=WGS84 +no_defs", "+proj=utm +zone=51 +datum=WGS84 +units=m +no_defs"
FIG_SIZE, FIG_DPI = (7,7), 150
GRID_RES_M = 100
VARIOGRAM_MODEL, RETRY_NUGGET = "spherical", 1e-3
COVERAGE_MAX_DIST_M = 3000

CMAP_VAR = mpl.colormaps["Greys"]  # lighter = lower uncertainty, darker = higher

def pick_pm_col(cols):
    for c in cols:
        if str(c).lower() in ("pm25","pm_25","pm2_5","pm2.5"): return c
    return None

# ---- Boundary & grid
with Env(SHAPE_RESTORE_SHX="YES"):
    pasig = gpd.read_file(PASIG_SHP)
if pasig.crs is None: pasig = pasig.set_crs(WGS84)
pasig_utm = pasig.to_crs(UTM51)
geom = unary_union(pasig_utm.geometry)
minx,miny,maxx,maxy = geom.bounds

xs = np.arange(minx, maxx+GRID_RES_M, GRID_RES_M)
ys = np.arange(miny, maxy+GRID_RES_M, GRID_RES_M)
grid_x, grid_y = np.meshgrid(xs, ys)

# ---- Files
files = sorted(glob.glob(os.path.join(DATA_DIR, "date_2025-*.gpkg")))
if not files: raise SystemExit("No daily GPKGs found.")

for f in files:
    base = os.path.splitext(os.path.basename(f))[0]
    day  = base.replace("date_","")

    g = gpd.read_file(f)
    if g.crs is None: g = g.set_crs(WGS84)

    if ("geometry" not in g) or g.geometry.is_empty.all():
        lon = next((c for c in g.columns if str(c).lower() in ("longitude","lon","x")), None)
        lat = next((c for c in g.columns if str(c).lower() in ("latitude","lat","y")), None)
        if lon and lat:
            g = g.set_geometry(gpd.points_from_xy(g[lon], g[lat], crs=g.crs))
        else:
            print("Skip (no geometry):", base); continue

    pm_col = pick_pm_col(g.columns)
    if pm_col is None: print("Skip (no PM):", base); continue

    pts = g.to_crs(UTM51)[[pm_col,"geometry"]].copy()
    pts["x"]=pts.geometry.x; pts["y"]=pts.geometry.y
    pts = pts.groupby(["x","y"], as_index=False)[pm_col].mean()
    pts = gpd.GeoDataFrame(pts, geometry=gpd.points_from_xy(pts["x"], pts["y"], crs=UTM51))

    x = pts["x"].to_numpy(); y = pts["y"].to_numpy()
    z = np.clip(pts[pm_col].astype(float).to_numpy(), 0, None)
    if len(z) < 3: print("Skip (n<3):", base); continue

    try:
        OK = OrdinaryKriging(x,y,z, variogram_model=VARIOGRAM_MODEL, enable_plotting=False,
                             verbose=False, coordinates_type="euclidean")
        zg, zv = OK.execute("grid", xs, ys, backend="loop")
        zv = np.array(zv)
    except LinAlgError:
        OK = OrdinaryKriging(x,y,z, variogram_model=VARIOGRAM_MODEL, nugget=RETRY_NUGGET,
                             enable_plotting=False, verbose=False, coordinates_type="euclidean")
        zg, zv = OK.execute("grid", xs, ys, backend="loop")
        zv = np.array(zv)

    # Masks: coverage + inside boundary
    tree = cKDTree(np.c_[x,y]); dists,_ = tree.query(np.c_[grid_x.ravel(), grid_y.ravel()], k=1)
    cover = (dists.reshape(grid_x.shape) <= COVERAGE_MAX_DIST_M)
    centers = gpd.GeoSeries(gpd.points_from_xy(grid_x.ravel(), grid_y.ravel()), crs=UTM51)
    inside  = centers.within(geom).to_numpy().reshape(zv.shape)
    var_mask = np.where(cover & inside, np.maximum(zv, 0.0), np.nan)

    # Robust normalization (95th pct) so extremes don't dominate
    vmax = float(np.nanpercentile(var_mask, 95)) if np.isfinite(var_mask).any() else 1.0
    norm = mpl.colors.Normalize(vmin=0.0, vmax=max(1e-9, vmax))

    fig, ax = plt.subplots(figsize=FIG_SIZE, dpi=FIG_DPI)
    im = ax.imshow(var_mask, extent=[xs[0], xs[-1], ys[0], ys[-1]],
                   origin="lower", cmap=CMAP_VAR, norm=norm)
    pasig_utm.boundary.plot(ax=ax, color="black", linewidth=0.8)
    cb = plt.colorbar(im, ax=ax, shrink=0.8); cb.set_label("Uncertainty (kriging variance)")
    ax.set_title(f"Uncertainty Map — {day}\nDarker = higher uncertainty")
    ax.set_axis_off()

    out = os.path.join(OUT_DIR, f"{base}_uncertainty.png")
    plt.tight_layout(); plt.savefig(out, bbox_inches="tight"); plt.close(fig)
    print("Saved:", out)


Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\uncertainty\date_2025-02-06_uncertainty.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\uncertainty\date_2025-02-07_uncertainty.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\uncertainty\date_2025-02-08_uncertainty.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\uncertainty\date_2025-02-09_uncertainty.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\uncertainty\date_2025-02-10_uncertainty.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\uncertainty\date_2025-02-11_uncertainty.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\uncertainty\date_2025-02-12_uncertainty.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\uncertainty\date_2025-02-13_uncertainty.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\uncertainty\date_2025-02-14_uncertainty.png
Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\uncertainty\date_2025-02